In [1]:
import tensorflow as tf
from tensorflow.config.experimental import list_logical_devices
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 3, 4"


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_memory_growth(gpus[1], True)
tf.config.experimental.set_memory_growth(gpus[2], True)

 
gpus = tf.config.experimental.list_logical_devices('GPU')

strategy = tf.distribute.MirroredStrategy([gpus[0].name, gpus[1].name, gpus[2].name])
print('\n\n Running on multiple GPUs ', [gpus[0].name, gpus[1].name, gpus[2].name])

2024-09-18 08:59:04.043321: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


 Running on multiple GPUs  ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2']


In [2]:
import os

# read image

disp_train_path = '/root/check/matching/datasets/US3D/experimental_data/disp/Track2-Truth/'

train_dataset_path = '/root/check/matching/datasets/US3D/experimental_data/left/Track2-RGB-1/'
all_left_paths, all_right_paths, all_disp_paths = [], [], []

for filename in os.listdir(train_dataset_path):
    if 'LEFT_RGB' in filename:
        left_path = os.path.join(train_dataset_path, filename)
        right_path = os.path.join(train_dataset_path, filename.replace('LEFT_RGB', 'RIGHT_RGB'))
        disp_path = os.path.join(disp_train_path, filename.replace('LEFT_RGB', 'LEFT_DSP'))
        
        all_left_paths.append(left_path)
        all_right_paths.append(right_path)
        all_disp_paths.append(disp_path)
    
val_dataset_path = '/root/check/matching/datasets/US3D/experimental_data/left/Track2-RGB-2/'
val_all_left_paths, val_all_right_paths, val_all_disp_paths = [], [], []

for filename in os.listdir(val_dataset_path):
    if 'LEFT_RGB' in filename:
        val_left_path = os.path.join(val_dataset_path, filename)
        val_right_path = os.path.join(val_dataset_path, filename.replace('LEFT_RGB', 'RIGHT_RGB'))
        val_disp_path = os.path.join(disp_train_path, filename.replace('LEFT_RGB', 'LEFT_DSP'))
        
        val_all_left_paths.append(val_left_path)
        val_all_right_paths.append(val_right_path)
        val_all_disp_paths.append(val_disp_path)
        

In [3]:
from math import exp
from tensorflow.keras.optimizers import Adam
from SRCVNet.srcvnet import *
from SRCVNet.data_reader import *
from SRCVNet.evaluation import *
from tensorflow.keras.callbacks import LearningRateScheduler

min_disp = -96
max_disp = 96

# Train the model
batch_size = 6
epochs = 100

learning_rate = 0.001


def learning_rate_decay(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch % 10 == 0:
        return lr * 0.5  # Explicitly cast to float
    else: 
        return lr

# Create the LearningRateScheduler callback
lr_decay_callback = LearningRateScheduler(learning_rate_decay)
# Compile the model

In [4]:
losses = {
    'estimation': sl1_8x,
    'estimation_1': sl1_4x,
    'refinement': sl1_1x,
    #'refinement_1': smooth_l1_loss_with_coefficient,
}

loss_weights = {
    'estimation': 0.6,
    'estimation_1': 1.0,
    'refinement': 0.7,
    #'refinement_1': 1.0,
}

In [5]:
epe = epe
d1 = d1

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Compile the model with the custom losses and metrics
with strategy.scope():
    net = SRCVNet(1024, 1024, 3, min_disp=min_disp, max_disp=max_disp)
    net.build_model()
    net.model.compile(optimizer=Adam(learning_rate = learning_rate),
                        loss=losses, 
                        loss_weights = loss_weights,
                        metrics=[epe, d1])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
feature_extraction (FeatureExtr [(None, 256, 256, 16 202128      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
cost_concatenation_1 (CostConca (None, 24, 128, 128, 0           feature_extraction[0][1]     

In [ ]:
import os
import random

train_generator = BatchLoader(all_left_paths, all_right_paths, all_disp_paths, batch_size=batch_size, reshuffle=True)
validation_generator = BatchLoader(val_all_left_paths, val_all_right_paths, val_all_disp_paths, batch_size=batch_size, reshuffle=False)

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
# Define a ModelCheckpoint callback

steps_per_epoch = len(train_generator) // batch_size


directory = "/root/check/matching/checkpoint/US3D/SRCVNet_woEASRCV/"

tboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=directory ) 


checkpoint_callback = ModelCheckpoint(
    os.path.join(directory, "SRCVNet_checkpoint.h5py"),
    save_best_only=True, monitor='val_refinement_epe',# Save only the best model
)
 
loss_history = []
accuracy_history = []

# Train the model with the callback
history = net.model.fit(train_generator, 
                        epochs=epochs, 
                        #steps_per_epoch = len(train_generator),
                        validation_data=validation_generator,
                        #validation_steps = len(validation_generator),
                        callbacks=[lr_decay_callback, checkpoint_callback, tboard_callback])

2024-09-18 08:59:20.982422: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2024-09-18 08:59:20.982471: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2024-09-18 08:59:20.982523: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 3 GPUs
2024-09-18 08:59:21.002072: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcupti.so.11.3
2024-09-18 08:59:21.438433: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2024-09-18 08:59:21.438982: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1744] CUPTI activity buffer flushed


. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto_shard_policy = AutoShardPolicy.DATA` before applying the options object to the dataset via `dataset.with_options(options)`.
2024-09-18 08:59:22.694997: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-09-18 08:59:22.717532: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2644800000 Hz


Epoch 1/100
